In [2]:
#셀레니움 기본 템플릿
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#데이터 추출 시 활용
from bs4 import BeautifulSoup
import time

#크롬 드라이버 생성
driver = webdriver.Chrome()

#페이지 이동
driver.get('https://map.naver.com/p/entry/place/1220132635?placePath=/home?entry=plt&from=map&fromPanelNum=1&additionalHeight=76&timestamp=202510141633&locale=ko&svcName=map_pcv5&searchType=place&lng=127.1320113&lat=37.4292607&c=15.00,0,0,0,dh')


문제 정의 : 긍정/부정 리뷰를 통해 보완점 찾기<br>
1.최신순으로 정렬<br>
2.무한 스크롤 처리<br>
3.id, 내용, 날짜, 시간, 방문 횟수, 키워드 수집

In [3]:
#iframe 제어
driver.switch_to.frame('entryIframe')

In [8]:
#리뷰 클릭
driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div.place_fixed_maintab.place_stuck > div > div > div > div > a:nth-child(3) > span').click()

In [4]:
#1. 최신순으로 정렬
# driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.k1QQ5 > div.place_section_content > div.fHbwT > div.place_option_area.mlywZ > a:nth-child(2)').click()

In [ ]:
#2. 무한 스크롤 처리

for i in range(1,25):
        if i == 1:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1)

        elif i in range (2,24):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight-1000)')
            time.sleep(1)
            try:
                more_tag = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.k1QQ5 > div.NSTUp > div > a > span')
                more_tag.click()
                time.sleep(1)
            except:
                print(f'{i}번째 더보기 버튼 없음')
                 

        else:
             driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
             time.sleep(1)

#keyword버튼 클릭
        buttons = driver.find_elements(By.CSS_SELECTOR,'.pui__jhpEyP.pui__ggzZJ8')
        for button in buttons:
             try:
                  driver.execute_script('arguments[0].click()', button)
                  time.sleep(0.2)
             except:
                  continue
             
             

In [10]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# id, 내용, 날짜, 시간, 방문 횟수, 키워드 수집

data = []

items = soup.select('.place_apply_pui.EjjAW')
for item in items:
    id = item.select_one('.pui__NMi-Dp').text
    content = item.select_one('.pui__vn15t2').text
    visit_dt = item.select_one('.pui__QKE5Pr > .pui__gfuUIT:nth-of-type(1)').contents[2].text
    visit_tm = item.select_one('.pui__ETqMYH').text
    visit_cnt = item.select_one('.pui__QKE5Pr > .pui__gfuUIT:nth-of-type(2)').text
    keyword = item.select_one('.pui__HLNvmI').text.split('요')
    # print(id, content, visit_dt, visit_tm, visit_cnt, keyword)
    data.append([id, content, visit_dt, visit_tm, visit_cnt, keyword])

data

In [ ]:
#excel파일로 저장

import pandas as pd
df = pd.DataFrame(data, columns=['id', 'content', 'visit_dt', 'visit_tm', 'visit_cnt', 'keyword'])
df.to_excel('../data/partyroom_C_review.xlsx', index = False)